In [1]:
import os
import glob
import random 
import itertools

import numpy as np
import pandas as pd

from pathlib import Path
from scipy import signal

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split, Subset

from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from esn_model import ESN, ReadOut

# ヘルパー
from rc_timeseries_helpers import (
    infer_device_from_model,
    extract_states_time_major,
    extract_logits_time_major,
    apply_time_selection,
    prepare_time_distributed_targets,
    compute_loss_time_kept,
    sequence_accuracy_majority_vote
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_encoder = preprocessing.LabelEncoder()

In [2]:
import wandb
# WandBの設定
WANDB_API_KEY = "2d996a98ef8dddefa91d675f85b5efd96fb911ae"  # あなたのWandB APIキーをここに入力してください

wandb.login(key = WANDB_API_KEY)

wandb: Currently logged in as: nekodaisuki169 (doctor_thesis_material). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/sota/.netrc


True

In [3]:
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import Dataset

class TactileSequenceDataset(Dataset):
    """
    触覚センサ npy データ用 Dataset クラス（CV/Stratify向け）

    - 入力ファイル: (T, n_taxels, 3)
    - [seq_start:seq_end] を切り出し -> (seq_len, n_taxels*3)
    - ESN向けに (1, seq_len, feature_dim) を返す
    - ラベルは one-hot ではなく「class index (long)」を返す（rc_timeseries_helpers が one-hot化する）
    """

    def __init__(self, root_dir, dataset_params):
        super().__init__()
        self.root_dir = Path(root_dir)
        self.seq_start = int(dataset_params["seq_start"])
        self.seq_end   = int(dataset_params["seq_end"])
        self.dtype     = torch.float32
        self.mmap_mode = dataset_params.get("mmap_mode", None)  # "r" 推奨（必要なら）

        # クラスディレクトリ列挙
        self.class_names = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        if not self.class_names:
            raise RuntimeError(f"No class directories under {self.root_dir}")

        self.class_to_idx = {c: i for i, c in enumerate(self.class_names)}
        self.num_classes  = len(self.class_names)

        # samples: (path, label_idx)
        self.samples = []
        for class_name in self.class_names:
            class_dir = self.root_dir / class_name
            for npy_path in sorted(class_dir.glob("*.npy")):
                self.samples.append((npy_path, self.class_to_idx[class_name]))

        if not self.samples:
            raise RuntimeError(f"No .npy files found under {self.root_dir}")

        # ★ これが StratifiedKFold 用に効く
        self.labels = [lab for _, lab in self.samples]

        # 先頭で形状チェック
        arr0 = np.load(self.samples[0][0], mmap_mode=self.mmap_mode)
        if arr0.ndim != 3:
            raise ValueError(f"Expected (T, n_taxels, 3), got shape={arr0.shape}")
        T, n_taxels, axes = arr0.shape
        if axes != 3:
            raise ValueError(f"Last dim must be 3, got {axes}")
        if self.seq_end > T:
            raise ValueError(f"seq_end({self.seq_end}) > T({T}). Please adjust.")

        self.original_T  = T
        self.n_taxels    = n_taxels
        self.seq_len     = self.seq_end - self.seq_start
        self.feature_dim = self.n_taxels * 3

        print("=== TactileSequenceDataset initialized ===")
        print(f"root_dir     : {self.root_dir}")
        print(f"num_classes  : {self.num_classes}")
        print(f"num_samples  : {len(self.samples)}")
        print(f"original T   : {self.original_T}")
        print(f"seq range    : [{self.seq_start}, {self.seq_end}) -> seq_len={self.seq_len}")
        print(f"n_taxels     : {self.n_taxels}")
        print(f"feature_dim  : {self.feature_dim} (= n_taxels * 3)")
        print("class_to_idx : ", self.class_to_idx)
        print("=========================================")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        npy_path, label_idx = self.samples[idx]

        arr = np.load(npy_path, mmap_mode=self.mmap_mode)  # (T, n_taxels, 3)
        seq = arr[self.seq_start:self.seq_end]             # (seq_len, n_taxels, 3)

        # (seq_len, n_taxels*3)
        seq = np.asarray(seq, dtype=np.float32).reshape(self.seq_len, self.feature_dim)

        # x: (1, seq_len, feature_dim)
        x = torch.from_numpy(seq).to(self.dtype).unsqueeze(0)

        # y: class index (long)
        y = torch.tensor(label_idx, dtype=torch.long)

        return x, y


In [4]:
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import StratifiedKFold

def _unwrap_subset(ds):
    """Subset(Subset(...)) でも base dataset と base index 列に展開して返す"""
    idxs = list(range(len(ds)))
    while isinstance(ds, Subset):
        idxs = [ds.indices[i] for i in idxs]
        ds = ds.dataset
    return ds, idxs

def _get_stratify_labels(ds):
    """
    StratifiedKFold 用 y を返す（dataset.labels を最優先）。
    labels が無ければ samples/targets から取る。最終手段は __getitem__ で推定（遅い）。
    """
    base, base_idxs = _unwrap_subset(ds)

    if hasattr(base, "labels"):
        base_labels = list(base.labels)
        return [base_labels[i] for i in base_idxs]

    if hasattr(base, "samples"):
        base_labels = [lab for _, lab in base.samples]
        return [base_labels[i] for i in base_idxs]

    if hasattr(base, "targets"):
        base_labels = list(base.targets)
        return [base_labels[i] for i in base_idxs]

    # fallback（遅い）：getitem から推定（y が index/one-hot どちらでもOK）
    labels = []
    for i in range(len(ds)):
        y = ds[i][1]
        if torch.is_tensor(y):
            if y.dtype.is_floating_point:  # one-hot
                labels.append(int(y.argmax().item()))
            else:  # index
                labels.append(int(y.view(-1)[0].item()))
        else:
            labels.append(int(y))
    return labels

def create_cross_validation_dataloaders(dataset, dataset_params, traing_params):
    """
    - dataset が Dataset / Subset / Subsetのネスト いずれでもOK
    - dataset.labels を最優先で stratify を作る
    - 正規化はしない（Datasetをそのまま読む）
    - “フルバッチ” ローダを返す（既存方針維持）
    """
    n_splits   = int(traing_params["n_splits"])
    seed       = int(traing_params.get("seed", dataset_params.get("seed", 0)))

    y = _get_stratify_labels(dataset)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    loaders = []
    for tr_idx, va_idx in skf.split(range(len(dataset)), y):
        tr = Subset(dataset, tr_idx)
        va = Subset(dataset, va_idx)

        # 既存の「フルバッチ」方針を維持（batch_size は無視して全件）
        loaders.append((
            DataLoader(tr, len(tr), shuffle=True),
            DataLoader(va, len(va), shuffle=False),
        ))

    return loaders


def prepare_datasets(dataset_params, traing_params, data_dir):
    testdata_ratio = float(traing_params["testdata_ratio"])
    batch_size = int(dataset_params["batch_size"])
    
    # データセットの準備
    dataset = TactileSequenceDataset(data_dir, dataset_params)  # 拡張はここでは適用しない

    # データセットを学習用、テスト用に分割する
    test_size = int(len(dataset) * testdata_ratio)
    train_size = len(dataset) - test_size
    crossval_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # 交差検証のためのデータローダーを準備
    cross_val_loaders = create_cross_validation_dataloaders(crossval_dataset, dataset_params, traing_params)

    # テストデータローダーの準備
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

    return cross_val_loaders, test_loader

In [17]:


# =========================
# 2) train_model / validate_model を丸ごと置き換え
# =========================
def train_model(model, criterion, optimizer, train_loader, dataset_params, model_params, traing_params):
    model.train()

    num_epochs = int(traing_params["num_epochs"])
    batch_training = bool(model_params["Batch_Training"])
    Regularization_L2 = float(model_params["Regularization_L2"])

    device = infer_device_from_model(model, fallback="cpu")

    washout_steps = int(dataset_params.get("washout_steps", 0))
    time_stride   = int(dataset_params.get("time_stride", 1))

    for epoch in range(num_epochs):
        loss_sum = 0.0
        acc_sum  = 0.0
        n_seen   = 0

        for inputs, labels in train_loader:
            inputs = inputs.float().to(device)
            labels = labels.to(device)

            # ---- forward ----
            esn_out = model.ESN(inputs)         # [B,1,T,H] or [B,T,H]
            ro_out  = model.ReadOut(esn_out)    # [B,1,T,C] or [B,T,C]

            # ---- normalize shapes to time-major ----
            states_BTH = extract_states_time_major(esn_out)  # [B,T,H]
            logits_BTC = extract_logits_time_major(ro_out)   # [B,T,C]

            if states_BTH.shape[1] != logits_BTC.shape[1]:
                raise ValueError(f"T mismatch: states T={states_BTH.shape[1]}, logits T={logits_BTC.shape[1]}")

            # ---- washout / stride ----
            states_BTH = apply_time_selection(states_BTH, washout_steps=washout_steps, time_stride=time_stride)
            logits_BTC = apply_time_selection(logits_BTC, washout_steps=washout_steps, time_stride=time_stride)

            B, T_eff, H = states_BTH.shape
            _, _, C = logits_BTC.shape

            # ---- loss (T kept) + targets for ridge ----
            loss, targets_onehot_BTC, target_index_B = compute_loss_time_kept(
                logits_time_major=logits_BTC,
                labels=labels,
                criterion=criterion
            )

            # ---- update ----
            if batch_training:
                # ridge update requires 2D: X [H, B*T], Y [C, B*T]
                X = states_BTH.permute(2, 0, 1).contiguous().view(H, -1)                  # [H, B*T]
                Y = targets_onehot_BTC.permute(2, 0, 1).contiguous().view(C, -1)          # [C, B*T]
                model.ReadOut.ridge_regression_update(X, Y, model, Regularization_L2)
            else:
                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                optimizer.step()

            # ---- metrics (sequence-level) ----
            acc = sequence_accuracy_majority_vote(logits_BTC, target_index_B)

            loss_sum += float(loss.item()) * B
            acc_sum  += float(acc) * B
            n_seen   += B

        epoch_loss = loss_sum / max(n_seen, 1)
        epoch_acc  = acc_sum  / max(n_seen, 1)

        print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc*100:.2f}%")
        try:
            wandb.log({"loss": epoch_loss, "train_accuracy": epoch_acc})
        except Exception:
            pass

    print("Training complete")


def validate_model(model, val_loader, dataset_params, model_params, criterion):
    model.eval()

    device = infer_device_from_model(model, fallback="cpu")
    washout_steps = int(dataset_params.get("washout_steps", 0))
    time_stride   = int(dataset_params.get("time_stride", 1))

    loss_sum = 0.0
    acc_sum  = 0.0
    n_seen   = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.float().to(device)
            labels = labels.to(device)

            esn_out = model.ESN(inputs)
            ro_out  = model.ReadOut(esn_out)

            states_BTH = extract_states_time_major(esn_out)
            logits_BTC = extract_logits_time_major(ro_out)

            if states_BTH.shape[1] != logits_BTC.shape[1]:
                raise ValueError(f"T mismatch: states T={states_BTH.shape[1]}, logits T={logits_BTC.shape[1]}")

            states_BTH = apply_time_selection(states_BTH, washout_steps=washout_steps, time_stride=time_stride)
            logits_BTC = apply_time_selection(logits_BTC, washout_steps=washout_steps, time_stride=time_stride)

            B = logits_BTC.shape[0]

            loss, _, target_index_B = compute_loss_time_kept(
                logits_time_major=logits_BTC,
                labels=labels,
                criterion=criterion
            )
            acc = sequence_accuracy_majority_vote(logits_BTC, target_index_B)

            loss_sum += float(loss.item()) * B
            acc_sum  += float(acc) * B
            n_seen   += B

    val_loss = loss_sum / max(n_seen, 1)
    val_acc  = acc_sum  / max(n_seen, 1)

    print(f"Validation | Loss: {val_loss:.4f} | Acc: {val_acc*100:.2f}%")
    try:
        wandb.log({"val_loss": val_loss, "val_accuracy": val_acc})
    except Exception:
        pass

    return val_loss, val_acc


 
    
def test_model(model, val_loader):
    model.eval()  # Set model to evaluation mode
    val_running_loss = 0.0
    val_running_corrects = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.float().to(device)
            labels = labels.squeeze().to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
#             _, label_preds = torch.max(labels, 1)
            label_preds = labels

            val_running_loss += loss.item() * inputs.size(0)
            print(loss.item())
            val_running_corrects += torch.sum(preds == label_preds)

    val_loss = val_running_loss / len(val_loader.dataset)
    val_accuracy = val_running_corrects.double() / len(val_loader.dataset)
    
    print(f'Test Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')
    wandb.log({"test_accuracy": val_accuracy, "test_loss": val_loss})

def compute_accuracy(model_output, target, n_taus):
    # モデルの出力と教師ラベルを各データに分割
    split_model_output = torch.split(model_output.squeeze(), n_taus, dim=-1)
    split_target = torch.split(target.squeeze(), n_taus, dim=-1)
    # print("aaaa")
    # print(split_model_output[0].shape)
    # print(split_target[0].shape)
    correct = 0
    total = 0
    
    for pred, true_label in zip(split_model_output, split_target):
        # 最も確率が高いラベルを予測ラベルとして取得
        # print(pred.shape)
        # print(pred)
        # print(true_label.shape)
        # print(true_label)
        count_ones = (true_label == 1).sum().item()
        # print(count_ones)
        histgram_predict = torch.bincount(torch.max(pred, 0)[1])
        _, predicted = torch.max(histgram_predict, 0)
    
        histgram_true_label_idx = torch.bincount(torch.max(true_label, 0)[1])
        _, true_label_idx = torch.max(histgram_true_label_idx, 0)

        # 正解数をカウント
        correct += (predicted == true_label_idx).sum().item()
        total += 1

    # 精度を算出
    accuracy = correct / total
    return accuracy

def model_params_candinate(model_params):
    model_params_combinations = list(itertools.product(*model_params.values()))
    param_dicts = [dict(zip(model_params.keys(), combination)) for combination in model_params_combinations]
    return param_dicts

# モデル構造を辞書型に格納
def model_sturcture_dict(model):
    layers_dict = {}
    for name, module in model.named_modules():
        layers_dict[name] = {
            'type': type(module).__name__,
            'parameters': {p: getattr(module, p) for p in module.__dict__ if not p.startswith('_')}
        }
    # モデル名と初期の引数は削除
    del(layers_dict[''])
    return layers_dict
    

In [13]:
from torch.utils.data import Subset

def base_indices(ds):
    idxs = list(range(len(ds)))
    while isinstance(ds, Subset):
        idxs = [ds.indices[i] for i in idxs]
        ds = ds.dataset
    return ds, idxs

def paths_from_loader(loader):
    base, idxs = base_indices(loader.dataset)
    return set(str(base.samples[i][0]) for i in idxs)

for fold, (tr, va) in enumerate(cross_val_loaders):
    inter = paths_from_loader(tr) & paths_from_loader(va)
    print(f"fold={fold} overlap_paths={len(inter)}")

fold=0 overlap_paths=0
fold=1 overlap_paths=0
fold=2 overlap_paths=0
fold=3 overlap_paths=0
fold=4 overlap_paths=0


In [18]:


#各種のパラメータ設定
dataset_params = {"seq_start" : 400, "seq_end" : 1200, "sequence_length": 800, "slicing_size" : 1, "augmentation_factor": 0, "batch_size" : 32, "Onehot_Encoding" : None, "augmentation_mu" : 0, "augmentation_sigma" : 0, "augmentation_shift" : 1, "time_stride":1}
model_params = {"reservoir_size" : [100],"input_size" : [48], "channel_size" : [1],  "reservoir_weights_scale" : [1], "input_weights_scale" : [10000000], "spectral_radius" : [1000],"reservoir_density" : [0.99], "leak_rate" : [0.99], "Batch_Training" : [True], "ReadOut_output_size" : [25], "Regularization_L2" : [0.0]}
training_params = {"num_epochs" : 1, "learning_rate" : 0.01, "weight_decay" : 1e-2, "testdata_ratio" : 0, "n_splits" : 5}

#それぞれのモデルパラメータ候補を辞書に格納する
model_params = model_params_candinate(model_params)

#学習データセットの設定
data_dir="./normalized_dataset/20251215_161803/"
cross_val_loaders, test_loader = prepare_datasets(dataset_params, training_params, data_dir)

# wandb.init(project="uskin_test_ESN", config=config_dictionary)

#モデルパラメータの候補ごとに，総当たりしてパラメータを探索する

for each_model_params in model_params:
    
#     model = LSTMModel(each_model_params).to(device)
    model = ESN(each_model_params, training_params, dataset_params).to(device)
    model_sturcture = model_sturcture_dict(model)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr = float(training_params["learning_rate"]), weight_decay = float(training_params["weight_decay"]))
    
    config_dictionary = {
    "dataset": data_dir,
    "dataset_params" : dataset_params,
    "architecture": model.__class__.__name__,
    "model_params" : each_model_params,
    "model_sturcture" : model_sturcture,
    "traing_params" : training_params,
    "criterion" : str(criterion),
    "optimizer" : str(optimizer),
    }

    wandb.init(project="uskin_test_NewESN", config=config_dictionary)

    # 4. k-fold交差検証のループ
    for fold, (train_loader, val_loader) in enumerate(cross_val_loaders):
        print(f'FOLD {fold}')
        print('--------------------------------')
        model = ESN(each_model_params, training_params, dataset_params).to(device)
        criterion = nn.MSELoss()
        train_model(model, criterion, optimizer, train_loader, dataset_params, each_model_params, training_params)
        validate_model(model, val_loader, dataset_params, each_model_params, criterion)
        # model.__init__(each_model_params, training_params, dataset_params)

    print('CrossVaridation Finished')
    print('--------------------------------')
    #テストデータによる評価
    # テストデータローダーの準備

#     test_model(model, test_loader)
    
    wandb.finish()

=== TactileSequenceDataset initialized ===
root_dir     : normalized_dataset/20251215_161803
num_classes  : 25
num_samples  : 250
original T   : 1255
seq range    : [400, 1200) -> seq_len=800
n_taxels     : 16
feature_dim  : 48 (= n_taxels * 3)
class_to_idx :  {'01_table_cover': 0, '02_fur_scarf': 1, '03_washing_towel': 2, '04_carpet1': 3, '05_bubble_wrap': 4, '06_fleece_scarf': 5, '07_knit_hat1': 6, '08_body_towel1': 7, '09_body_towel2': 8, '10_carpet2': 9, '11_work_gloves': 10, '12_knit_hat2': 11, '13_toilet_mat1': 12, '14_floor_mat': 13, '15_sponge1': 14, '16_printed_tatami': 15, '17_cushion1': 16, '18_mop': 17, '19_toilet_mat2': 18, '20_fleece_sock': 19, '21_cushion': 20, '22_carpet3': 21, '23_fleece_mat': 22, '24_carpet4': 23, '25_sponge2': 24}


loss,▁
train_accuracy,▁
loss,1.42836
train_accuracy,0.05


FOLD 0
--------------------------------


ValueError: not enough values to unpack (expected 2, got 1)